In [19]:
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import pandas as pd
import numpy as np

data_folder = "./data/"

#Get and load data
# pitch_14_17_file = "pitcher_2014_2017.csv"
bball_data_2_file = "Baseball Data-2.csv"

In [6]:
#preprocessing function used to calculate the plate location and isStrike classifier (this will be different than PitchCall (even in terms of BallCalled vs StrikeCalled))
#returns tuple: classification, isStrike
#classification: heart (strike) = 0, shadow (strike) = 1, shadow (ball) = 2, chase (ball) = 3, waste (ball) = 4
def PlateZone(PlateLocHeight, PlateLocSide):
    foot = 12

    #waste outside 84in to 6in, -20in to 20in horizontal, (strike zone * 200%)
    if ((PlateLocHeight > 7 or PlateLocHeight < 0.5) and (PlateLocSide < -(20/foot) or PlateLocSide > (20/foot))):
        return 4

    #heart inside 38in to 22in vertical, -6.7in to 6.7in horizontal, (strike zone size * 67%)
    if ((PlateLocHeight < (38/foot) and PlateLocHeight > (22/foot)) and (PlateLocSide > (-6.7/foot) and PlateLocSide < (6.7/foot))):
        return 0

    #strike zone inside 42in to 18in vertical, -10in to 10in horizontal
    if (PlateLocHeight < (42/foot) and PlateLocHeight > (18/foot) and (PlateLocSide > (-10/foot) and PlateLocSide < (10/foot))):
        return 1

    #shadow inside 46in to 14in vertical, -13.3in to 13.3in horizontal, (strike zone size * 133%)
    if (PlateLocHeight < (46/foot) and PlateLocHeight > (14/foot) and (PlateLocSide > (-13.3/foot) and PlateLocSide < (13.3/foot))):
        return 2

    #chase inside 84in to 6in, -20in to 20in horizontal, (strike zone * 200%)
        return 3

    #preprocessing function used to generate a single number that will be used to classify the ball/strike count before the current pitch 
    #returns int [0 - 11]
def PitchCount(balls, strikes):
    # Strikes: 0  1   2
    # Balls v|---------- 
    #       0| 0  1   2 
    #       1| 3  4   5
    #       2| 6  7   8
    #       3| 9  10  11
    if(balls == 0):
        if(strikes == 0):
            return 0
        if (strikes == 1):
            return 1
        return 2
    if(balls == 1):
        if(strikes == 0):
            return 3
        if (strikes == 1):
            return 4
        return 5
    if(balls == 2):
        if(strikes == 0):
            return 6
        if (strikes == 1):
            return 7
        return 8
    if(balls == 3):
        if(strikes == 0):
            return 9
        if (strikes == 1):
            return 10
        return 11

    #this preprocessing function generates the ground truth hitability of a pitch
    #these values will definitely need to be adjusted and should probably be hyperparameters
def GenerateGroundTruthLabels(pitchCall):
    if pitchCall == 'BallCalled':
        return -1
    if pitchCall == 'BallIntentional' or pitchCall == 'HitByPitch':
        return 0
    if pitchCall == 'StrikeSwinging' or pitchCall == 'StrikeCalled':
        return 1
    else:
        return 2


In [23]:
#import the columns we will need for training and preprocessing
bball_data = pd.read_csv(data_folder+bball_data_2_file, usecols=['Pitcher', 'PitcherThrows', 'Batter', 'BatterSide', 'PitchCall', 'KorBB', 'PlayResult', 'RunsScored', 'VertBreak', 'HorzBreak', 'ZoneSpeed', 'VertApprAngle', 'HorzApprAngle', 'ZoneTime', 'PlateLocHeight', 'PlateLocSide', 'Balls', 'Strikes', 'TaggedPitchType'])
#run preprocessing functions
bball_data['Zone'] = bball_data.apply(lambda pitch : PlateZone(pitch['PlateLocHeight'], pitch['PlateLocSide']), axis=1)
bball_data['BallStrikeNum'] = bball_data.apply(lambda pitch : PitchCount(pitch['Balls'], pitch['Strikes']), axis=1)
bball_data['GroundTruth'] = bball_data.apply(lambda pitch : GenerateGroundTruthLabels(pitch['PitchCall']), axis=1)

#drop all features we no longer need (ones only used for preprocessing)
bball_data.drop(labels=['PlateLocHeight', 'PlateLocSide', 'Balls', 'Strikes'], axis=1)


categorical_columns = ['Pitcher', 'Batter', 'PitchCall', 'KorBB', 'PlayResult', 'Zone', 'BallStrikeNum']
ohe_bball_data = OneHotEncoder(sparse=False)

for category in categorical_columns:
    bball_data[category] = bball_data[category].astype('category')
    ohe_bball_data.fit_transform(bball_data[[category]])

#print the labels of the features we will use to train our network
print(bball_data.keys())
print(ohe_bball_data.categories_)

#splitting data into training, validation, testing
total_samples = len(bball_data.index)
training_samples = math.floor(0.6*total_samples)
validation_samples = math.ceil(0.2*total_samples)
testing_samples = math.ceil(0.2*total_samples)

sum = training_samples+testing_samples+validation_samples

print("total samples:",total_samples,
        "\ntraining samples:",training_samples,
        "\nvalidation samples:",validation_samples,
        "\ntesting samples:",testing_samples,
        "\nsum of training, validation, and test samples:",sum)

#makes shuffled version of the data
indices = np.arange(total_samples)
np.random.shuffle(indices) #TODO get ground truth for data sets using these indices
shuffled_bball_data = bball_data.reindex(indices).reset_index()

#gets the amount of random data points as determined by set proportion
training_data = shuffled_bball_data.iloc[0:training_samples]
validation_data = shuffled_bball_data.iloc[training_samples:training_samples+validation_samples]
testing_data = shuffled_bball_data.iloc[training_samples+validation_samples:training_samples+validation_samples+testing_samples]

training_data

MemoryError: Unable to allocate 44.8 GiB for an array with shape (1040373, 5783) and data type float64

In [12]:
X = training_data.loc[:,~training_data.columns.isin(['GroundTruth'])]
y = training_data.loc[:,training_data.columns == 'GroundTruth']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

ValueError: could not convert string to float: 'Ellington, Eli'